# PyCity Schools Analysis

For this analysis, data was gathered from 15 different schools to report student performances and determine if students' grade, school type, school size, or per capita budget had any effect on those performances.  

One significant finding of this analysis was that per capita budget (i.e. spending per student) had an inverse relationship to student performance.  Schools that had lower spending per student generally reported higher student performances than schools with higher spending per student.

This analysis also demonstrated that small and medium schools (student populations of 2000 or less) reported higher student performances than large schools (student populations above 2000).

The last conclusion to be drawn from this analysis was that charter schools reported much higher student performances than district schools.

In [1]:
import pandas as pd
from pathlib import Path

schools_data = Path('Resources/schools_complete.csv')
students_data = Path('Resources/students_complete.csv')

schools_df = pd.read_csv(schools_data)
students_df = pd.read_csv(students_data)

merged_df = pd.merge(students_df, schools_df, how='left', on=['school_name', 'school_name'])
merged_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [2]:
# Calculating the total number of schools
# .value_counts() gives a list of all the unique school names and their occurrences in the data frame
# .count() provides the total number of unique school names obtained from .value_count()
total_schools = merged_df['school_name'].value_counts().count()
total_schools

15

In [3]:
# Calculating the total number of students
# .value_counts() gives a list of all the unique student ID numbers and their occurrences in the data frame
# .count() provides the total number of unique student ID numbers obtained from .value_count()
total_students = merged_df['Student ID'].value_counts().count()
total_students

39170

In [4]:
# Calculating the total budget
total_budget = schools_df['budget'].sum()
total_budget

24649428

In [5]:
# Calculating the average math score
avg_math = merged_df['math_score'].mean()
avg_math

78.98537145774827

In [6]:
# Calculating the average reading score
avg_reading = merged_df['reading_score'].mean()
avg_reading

81.87784018381414

In [7]:
# Percentage of students who passed math
pass_math_count = merged_df[(merged_df["math_score"] >= 70)].count()["student_name"]
pass_math_percent = pass_math_count / float(total_students) * 100
pass_math_percent

74.9808526933878

In [8]:
# Percentage of students who passed reading
pass_reading_count = merged_df[(merged_df["reading_score"] >= 70)].count()["student_name"]
pass_reading_percent = pass_reading_count / float(total_students) * 100
pass_reading_percent

85.80546336482001

In [9]:
# Percentage of students who passed both math and reading
pass_math_count = merged_df[
    (merged_df["math_score"] >= 70) & (merged_df["reading_score"] >= 70)
].count()["student_name"]
overall_pass_percent = pass_math_count /  float(total_students) * 100
overall_pass_percent

65.17232575950983

In [10]:
# Creates a series of items that reference our previously calculated values
district_summary_items = {
    'Total Schools': [total_schools],
    'Total Students': [total_students],
    'Total Budget': [total_budget],
    'Average Math Score': [avg_math],
    'Average Reading Score': [avg_reading],
    '% Passing Math': [pass_math_percent],
    '% Passing Reading': [pass_reading_percent],
    '% Overall Passing': [overall_pass_percent]
    }

# Creates a new data frame from the previously calculated values
district_summary = pd.DataFrame(district_summary_items, columns=[
    'Total Schools',
    'Total Students',
    'Total Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing'])

# Provides formatting for our Total Students and Total Budget columns
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Displays the new data frame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary

In [11]:
# Provides a list of each school's name and whether it is a district or charter school
school_types = schools_df.set_index(["school_name"])["type"]

In [12]:
# Provides the total student count for each school
per_school_counts = schools_df.set_index(["school_name"])["size"]

In [13]:
# Provides the budget for each school, then calculates the amount of budget allocated to each student
per_school_budget = schools_df.set_index(["school_name"])["budget"]
per_school_capita = per_school_budget / per_school_counts

In [14]:
# Calculates the average math scores for each school
per_school_math = merged_df.set_index(["school_name"])["math_score"]
per_school_math = per_school_math.groupby("school_name").sum()
per_school_math = per_school_math / per_school_counts

In [15]:
# Calculates the average reading scores for each school
per_school_reading = merged_df.set_index(["school_name"])["reading_score"]
per_school_reading = per_school_reading.groupby("school_name").sum()
per_school_reading = per_school_reading / per_school_counts

In [16]:
# Calculates the number of students with math scores of 70 or higher for each school
students_passing_math = merged_df.loc[(merged_df['math_score'] >= 70),:]
schools_students_passing_math = students_passing_math[['school_name', 'math_score']]
schools_students_passing_math = schools_students_passing_math.groupby(['school_name']).size()

In [17]:
# Calculates the number of students with reading scores of 70 or higher for each school
students_passing_reading = merged_df.loc[(merged_df['reading_score'] >= 70),:]
schools_students_passing_reading = students_passing_reading[['school_name', 'reading_score']]
schools_students_passing_reading = schools_students_passing_reading.groupby(['school_name']).size()

In [18]:
# Calculates the number of students with math and reading scores of 70 or higher for each school
students_passing_math_and_reading = merged_df[
    (merged_df["reading_score"] >= 70) & (merged_df["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [19]:
# Calculates passing percentages
per_school_passing_math = schools_students_passing_math / per_school_counts * 100
per_school_passing_reading = schools_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

In [20]:
# Gives names to the series of data we calculated previously so that they can be merged
school_types = school_types.rename('School Type')
per_school_counts = per_school_counts.rename('Total Students')
per_school_budget = per_school_budget.rename('Total School Budget')
per_school_capita = per_school_capita.rename('Per Student Budget')
per_school_math = per_school_math.rename('Average Math Score')
per_school_reading = per_school_reading.rename('Average Reading Score')
per_school_passing_math = per_school_passing_math.rename('% Passing Math')
per_school_passing_reading = per_school_passing_reading.rename('% Passing Reading')
overall_passing_rate = overall_passing_rate.rename('% Overall Passing')

# Merges all of our calculated values into a single DataFrame
per_school_summary = pd.merge(school_types, per_school_counts, left_index=True, right_index=True)
per_school_summary = pd.merge(per_school_summary, per_school_budget, left_index=True, right_index=True)
per_school_summary = pd.merge(per_school_summary, per_school_capita, left_index=True, right_index=True)
per_school_summary = pd.merge(per_school_summary, per_school_math, left_index=True, right_index=True)
per_school_summary = pd.merge(per_school_summary, per_school_reading, left_index=True, right_index=True)
per_school_summary = pd.merge(per_school_summary, per_school_passing_math, left_index=True, right_index=True)
per_school_summary = pd.merge(per_school_summary, per_school_passing_reading, left_index=True, right_index=True)
per_school_summary = pd.merge(per_school_summary, overall_passing_rate, left_index=True, right_index=True)

# Sorts DataFrame alphabetically by school name
# Provides formatting to the Total School Budget and Per Student Budget columns
per_school_summary = per_school_summary.sort_index()
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary.index.name = None

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Highest-Performing Schools (by % Overall Passing)

In [21]:
# Sorts Schools by % Overall Passing in descending order, then displays top 5 rows
top_schools = per_school_summary.sort_values(by=['% Overall Passing'], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Bottom Performing Schools (by % Overall Passing)

In [22]:
# Sorts Schools by % Overall Passing in ascending order, then displays top 5 rows
bottom_schools = per_school_summary.sort_values(by=['% Overall Passing'])
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


# Math Scores By Grade

In [23]:
math_df = merged_df[['school_name', 'grade', 'math_score']]

# Separates all of the school's data by grade
ninth_graders = math_df[(math_df["grade"] == "9th")]
tenth_graders = math_df[(math_df["grade"] == "10th")]
eleventh_graders = math_df[(math_df["grade"] == "11th")]
twelfth_graders = math_df[(math_df["grade"] == "12th")]

# Groups by 'school_name' and takes the mean of the 'math_score' for each school
# Renames the math_score column for each grade so they can be distinct in the final DataFrame
ninth_graders_math_scores = ninth_graders.groupby(['school_name']).mean(numeric_only=True).rename(columns={'math_score': '9th'})
tenth_graders_math_scores = tenth_graders.groupby(['school_name']).mean(numeric_only=True).rename(columns={'math_score': '10th'})
eleventh_graders_math_scores = eleventh_graders.groupby(['school_name']).mean(numeric_only=True).rename(columns={'math_score': '11th'})
twelfth_graders_math_scores = twelfth_graders.groupby(['school_name']).mean(numeric_only=True).rename(columns={'math_score': '12th'})

# Combines the scores above into a single DataFrame
math_scores_by_grade = pd.concat([ninth_graders_math_scores,
                                  tenth_graders_math_scores,
                                  eleventh_graders_math_scores,
                                  twelfth_graders_math_scores],
                                axis=1)
# Formatting
math_scores_by_grade.index.name = None

# Displays the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Score by Grade

In [24]:
reading_df = merged_df[['school_name', 'grade', 'reading_score']]

# Separates all of the school's data by grade
ninth_graders = reading_df[(reading_df["grade"] == "9th")]
tenth_graders = reading_df[(reading_df["grade"] == "10th")]
eleventh_graders = reading_df[(reading_df["grade"] == "11th")]
twelfth_graders = reading_df[(reading_df["grade"] == "12th")]

# Groups by 'school_name' and takes the mean of the 'math_score' for each school
# Renames the reading_score column for each grade so they can be distinct in the final DataFrame
ninth_graders_reading_scores = ninth_graders.groupby(['school_name']).mean(numeric_only=True).rename(columns={'reading_score': '9th'})
tenth_graders_reading_scores = tenth_graders.groupby(['school_name']).mean(numeric_only=True).rename(columns={'reading_score': '10th'})
eleventh_grader_reading_scores = eleventh_graders.groupby(['school_name']).mean(numeric_only=True).rename(columns={'reading_score': '11th'})
twelfth_grader_reading_scores = twelfth_graders.groupby(['school_name']).mean(numeric_only=True).rename(columns={'reading_score': '12th'})

# Combines the scores above into a single DataFrame
reading_scores_by_grade = pd.concat([ninth_graders_reading_scores,
                                  tenth_graders_reading_scores,
                                  eleventh_grader_reading_scores,
                                  twelfth_grader_reading_scores],
                                axis=1)
# Formatting
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Displays the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

In [25]:
# Establishing bins
spending_bins = [0, 585, 630, 645, 680]
labels = ['<$585', '$585-630', '$630-645', '$645-680']

In [26]:
# Creates a copy of our previous school summary
school_spending_df = per_school_summary.copy()

In [27]:
# Converts the Per Student Budget column to int data type so that we can properly cut and bin the column
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].str.replace('.00', '').str.replace('$', '').astype(int)

In [28]:
# Using pd.cut to categorize the data in our DataFrame based on the established bins
school_spending_df['Spending Ranges (Per Student)'] = pd.cut(school_spending_df['Per Student Budget'], 
                                                             bins=spending_bins, labels=labels)

# Reapplies the formatting to the Per Student Budget column
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].map("${:,.2f}".format)

school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,$585-630


In [29]:
# Calculates the averages for the desired columns
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=False)["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=False)["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=False)["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=False)["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=False)["% Overall Passing"].mean()

In [30]:
# Combines the above calculations into a single DataFrame
spending_summary = pd.merge(spending_math_scores, spending_reading_scores, left_index=True, right_index=True)
spending_summary = pd.merge(spending_summary, spending_passing_math, left_index=True, right_index=True)
spending_summary = pd.merge(spending_summary, spending_passing_reading, left_index=True, right_index=True)
spending_summary = pd.merge(spending_summary, overall_passing_spending, left_index=True, right_index=True)

spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


# Scores by School Size

In [31]:
# Defines the bins and lables
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [32]:
# Creates a copy of the previous school summary
school_size_df = per_school_summary.copy()

In [33]:
# Categorizes the schools as different sizes based on the bins
school_size_df['School Size'] = pd.cut(school_spending_df['Total Students'], 
                                                             bins=size_bins, labels=labels)

school_size_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


In [34]:
# Calculates averages for the desired columns
size_math_scores = school_size_df.groupby(["School Size"], observed=False)["Average Math Score"].mean()
size_reading_scores = school_size_df.groupby(["School Size"], observed=False)["Average Reading Score"].mean()
size_passing_math = school_size_df.groupby(["School Size"], observed=False)["% Passing Math"].mean()
size_passing_reading = school_size_df.groupby(["School Size"], observed=False)["% Passing Reading"].mean()
size_overall_passing = school_size_df.groupby(["School Size"], observed=False)["% Overall Passing"].mean()

In [35]:
# Creates a DataFrame that reports the school performance based on school size
size_summary = pd.merge(size_math_scores, size_reading_scores, left_index=True, right_index=True)
size_summary = pd.merge(size_summary, size_passing_math, left_index=True, right_index=True)
size_summary = pd.merge(size_summary, size_passing_reading, left_index=True, right_index=True)
size_summary = pd.merge(size_summary, size_overall_passing, left_index=True, right_index=True)

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


# Scores by School Type

In [36]:
# Groups the per_school_summary DataFrame by School Type and provides the averages for the desired columns
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [37]:
# Creates a DataFrame that reports school performance based on school type (i.e. district vs. charter)
type_summary = pd.merge(average_math_score_by_type, average_reading_score_by_type, left_index=True, right_index=True)
type_summary = pd.merge(type_summary, average_percent_passing_math_by_type, left_index=True, right_index=True)
type_summary = pd.merge(type_summary, average_percent_passing_reading_by_type, left_index=True, right_index=True)
type_summary = pd.merge(type_summary, average_percent_overall_passing_by_type, left_index=True, right_index=True)

type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
